# DD-CMA / Multi-Fidelity / Constraint Handling 

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt

from ddcma import DdCma, Checker, Logger
from constraint_handling import McrArchConstraintHandler
from compliance_minimization import ComplianceMinimization, NGNet
from adaptive_simulator_switcher import AbstractSimulator, AdaptiveSimulationSwitcher

In [2]:
class NGNetComplianceMinimization(AbstractSimulator):
    def __init__(self, i):
        """
        Parameters
        ----------
        n_basis_x, n_basis_y : int
            number of bases on each coordinate, n_basis_x * n_basis_y bases in total
        nelx, nely : int
            number of elements in each coordinate
        scale : float
            scale parameter of the Gaussian kernel
        volfrac : float, in [0, 1]
            total amount of material, V(x), is constrained to volfrac * V(1)
        eps : float, default 1e-3
            tolerance of equality constraint
        """
        self.nelx=12
        self.nely=6
        self.ng_x = 24
        self.ng_y = 12
        self.dim = self.ng_x * self.ng_y
        self.volfrac = 0.6
        self.co = ComplianceMinimization(self.nelx * i, self.nely * i, self.volfrac, penal=1.)
        self.ngnet = NGNet(self.ng_x, self.ng_y, self.nelx * i, self.nely * i, scale=1.0)
        
    def __call__(self, solution):
        """Compute the compliance and volume fraction
        
        Parameter
        ---------
        solution : object
            solution._x_repaired (input, 1d array-like) : height vector of NGNet, constrained in [-1, 1]
            solution.compliance (output, float)         : compliance
            solution.volution_fraction (output, float)  : volume fraction
            solution._f (output, float)                 : f(x)
            solution._g (output, 1d array-like)         : [g(x)]
        """
        fx, gx, _, _ = self.co(self.ngnet(solution._x_repaired))
        solution.compliance = fx
        solution.volume_fraction = gx
        solution._f = fx
        solution._g = [gx - self.volfrac]

In [3]:
class Solution:
    def __init__(self, x=None):
        self._x = x
        self._x_repaired = None
        self._f = None
        self._quak_penalty = None
        self._quak_violation = []
        self._qrsk_violation = []
    def clone(self):
        cl = Solution()
        cl._x = np.array(self._x, copy=True)
        cl._x_repaired = np.array(self._x_repaired, copy=True)
        cl._f = self._f
        cl._quak_penalty = self._quak_penalty
        cl._quak_violation = np.array(self._quak_violation, copy=True)
        cl._qrsk_violation = np.array(self._qrsk_violation, copy=True)
        return cl

In [4]:
# number of simulators
m = 5
# list of different approximate functions
func_list = [NGNetComplianceMinimization(2**i) for i in range(0, m)]
# Create the switcher (function wrapper)
adaptf = AdaptiveSimulationSwitcher(func_list) 

In [5]:
# Upper and Lower Bounds
N = adaptf.simulator_list[0].dim
lbound = -1.0 * np.ones(N)
ubound = 1.0 * np.ones(N)

# DD-CMA
ddcma = DdCma(xmean0=(lbound + ubound)/2. + (ubound - lbound)/4.,
            sigma0=(ubound - lbound)/4., 
            flg_variance_update=True, 
            flg_covariance_update=True,
            flg_active_update=True)

# ARCH + MCR Constraint Handling
# volume_fraction can be treated as either inequality or equality qrsk constraint.
ch = McrArchConstraintHandler(dim=ddcma.N, 
                            weight=ddcma.w, 
                            fobjective=adaptf.f, 
                            bound=(lbound, ubound),
                            linear_ineq_quak=None, 
                            linear_eq_quak=None, 
                            nonlinear_ineq_quak_list=None, 
                            nonlinear_eq_quak_list=None,
                            ineq_qrsk_list=[adaptf.get_gi(0)], 
                            eq_qrsk_list=None
                            )

# Termination Checker and Logger
checker = Checker(ddcma)
logger = Logger(ddcma, prefix='ddcma')

In [6]:
results = 'test.txt'
with open(results, 'w') as f:
    pass
# Execution
issatisfied = False
fbestsofar = np.inf
t_total = 0.0
while not issatisfied:
    #
    t_start = time.perf_counter()
    #
    xx, yy, zz = ddcma.sample()
    sol_list = [Solution(x) for x in xx]
    xcov = ddcma.transform(ddcma.transform(np.eye(N)).T)
    ch.prepare(ddcma.xmean, xcov)
    for sol in sol_list:
        ch.repair_quak(sol)
    adaptf.batcheval(sol_list)        
    for sol in sol_list:
        ch.evaluate_f_and_qrsk(sol)
    ranking = ch.total_ranking(sol_list)
    idx = np.argsort(ranking)
    ddcma.update(idx, xx, yy, zz)
    adaptf.update(sol_list, ch.do)
    #
    t_end = time.perf_counter()
    t_total += t_end - t_start
    with open(results, 'a') as f:
        f.write(repr(t_total) + ' ' + str(adaptf.idx_current) + ' ' 
                + repr(sol_list[idx[0]]._f) + ' ' + repr(sol_list[idx[0]]._qrsk_violation[0]) + ' '
                + ' '.join([repr(x) for x in sol_list[idx[0]]._x_repaired])
                + '\n')
    print(adaptf.idx_current, t_total, sol_list[idx[0]]._f, sol_list[idx[0]]._qrsk_violation[0])
    #
    ddcma.t += 1        
    ddcma.neval += ddcma.lam        
    ddcma.arf = np.array([sol._f for sol in sol_list])
    ddcma.arx = np.array([sol._x for sol in sol_list])
    fbest = ddcma.arf[idx[0]]
    fbestsofar = min(fbest, fbestsofar)
    if fbest < -np.inf:
        issatisfied, condition = True, 'ftarget'
    elif ddcma.t > 1000:
        issatisfied, condition = True, 'maxiter'
    else:
        pass
        #issatisfied, condition = checker()
    if ddcma.t % 10 == 0:
        print(ddcma.t, ddcma.neval, fbest, fbestsofar)
        logger()
print(ddcma.t, ddcma.neval, fbest, fbestsofar)
print("Terminated with condition: " + condition)
logger(condition)

0 0.3223107820000024 46.206741488545774 0.3861111111111112
0 0.46755195700000485 43.65998820669344 0.3861111111111112
0 1.0151191590000082 56.60528650570985 0.37222222222222223
0 1.2309952090000067 47.26315299627896 0.3583333333333334
0 1.8135680310000097 46.97401098881748 0.3583333333333334
0 2.2769419120000123 46.97401098881748 0.3583333333333334
0 2.463584111000017 50.888715502387754 0.34444444444444444
0 2.9068660060000155 60.28208527408843 0.31666666666666665
0 3.0627958120000187 61.27839291491873 0.3027777777777778
0 3.509020755000023 90.76469986992491 0.275
10 200 90.76469986992491 43.65998820669344
0 3.669979359000024 2022222334.124936 0.2333333333333334
0 4.210061582000023 552.570912763556 0.21944444444444444
0 4.77893488700002 3033338711.026297 0.2055555555555556
0 4.978891462000014 2022222557.6398997 0.15000000000000002
0 5.184331730000011 4530336180.785404 0.13611111111111118
0 5.737923118000012 4923981874.079728 0.10833333333333339
0 5.915001661000012 6878874937.54512 0.05

2 86.956774534 120.5590783484246 0.0032986111111111827
130 2600 120.5590783484246 43.65998820669344
2 87.76165627 120.94063466026853 0.0015625000000000222
2 89.472183345 116.30772169379244 0.005902777777777812
2 90.22068741700001 114.82876919977649 0.004166666666666652
2 90.94275243400001 109.81917879093622 0.0067708333333333925
2 91.70024857500002 115.6689794129816 0.0032986111111111827
2 92.41695918700002 120.35068347379641 0.0024305555555556024
2 94.15223092200002 111.44471544967453 0.010243055555555602
2 94.88896083600001 113.78452695728211 0.005034722222222232
2 95.65731086500003 109.63445344245514 0.000694444444444442
2 97.60019680300005 118.42667596272094 0.000694444444444442
140 2800 118.42667596272094 43.65998820669344
2 98.31635720500003 110.81402429537837 0.0015625000000000222
2 99.00946165400003 108.94788758120374 0.000694444444444442
2 99.74363814200001 109.36192432539903 0.000694444444444442
2 100.45274148800002 108.81168013430764 0.0024305555555556024
2 102.0904023140000

2 195.83127254899992 76.97769663054024 0.008506944444444442
2 197.33543008999993 77.31574369330019 0.004166666666666652
2 197.88273449999994 78.13822422449036 0.0024305555555556024
260 5200 78.13822422449036 43.65998820669344
2 198.42724968399995 77.47965011251894 0.005902777777777812
2 198.97356210099997 76.82937072114888 0.000694444444444442
2 199.51033291899998 77.30163268192878 0.005034722222222232
2 201.03073358799998 79.418749860304 0.0024305555555556024
2 201.57686577599998 76.89064545697192 0.004166666666666652
2 202.12840817999998 77.37554121707802 -0.0010416666666666075
2 202.697116564 76.59833562080826 0.0032986111111111827
2 204.21279647900002 76.54625425241505 0.005034722222222232
2 204.75462160400002 77.35798777646234 0.0032986111111111827
2 205.423017605 76.55399247773094 0.0067708333333333925
270 5400 76.55399247773094 43.65998820669344
2 206.02257523100002 76.25654240585828 0.008506944444444442
2 208.09729682800003 76.60021297444054 0.005902777777777812
2 209.099629074

3 415.145855216 78.0725137238818 0.000694444444444442
3 416.565525452 77.71116227471543 0.0015625000000000222
3 422.33296484799996 78.13186108510631 0.000694444444444442
3 423.7368704809999 77.69876812595052 0.001128472222222232
390 7800 77.69876812595052 43.65998820669344
3 425.1491764149999 77.77420805062096 0.001128472222222232
3 426.60312852099986 78.49176140771561 0.0009114583333333925
3 428.05861502399983 77.0413286752453 0.0067708333333333925
3 433.80689380899986 78.2040980523636 0.00047743055555560243
3 435.22685892099986 78.3940886901027 0.00026041666666665186
3 436.67096371699984 77.98809046720828 0.000694444444444442
3 438.10650741499984 77.97683544652132 0.0013454861111111827
3 443.84805546799987 77.65131611878891 0.0015625000000000222
3 445.2521605189999 78.88390125781545 0.00026041666666665186
3 446.7020791879999 77.234434337215 0.006336805555555602
400 8000 77.234434337215 43.65998820669344
3 448.1279952669999 77.19562315008113 0.001128472222222232
3 453.9596710589999 77

3 731.3327016789996 76.06124190070773 0.0015625000000000222
3 732.7765466709996 75.85504614784503 0.001128472222222232
3 738.9795787549995 76.06490068027361 0.0013454861111111827
3 740.9253492849995 76.275400002608 0.0009114583333333925
3 742.5088185199994 76.25515700122915 0.000694444444444442
3 744.0674694509994 76.02517569461062 0.001128472222222232
520 10400 76.02517569461062 43.65998820669344
3 750.0398885179993 76.31137912750755 4.340277777781232e-05
3 751.6009094239994 76.35906914277604 0.0017795138888888618
3 753.0009675149994 76.6277480012794 4.340277777781232e-05
3 754.4472828159993 76.64563741096704 0.00026041666666665186
3 755.8532370759992 75.58471886129624 0.006336805555555602
3 761.5816254639993 76.13380527829189 0.0019965277777778123
3 763.0229660009993 76.4214635342727 0.00026041666666665186
3 764.5218437449993 76.53865551455573 4.340277777781232e-05
3 766.0173691389992 76.12670772973837 0.002213541666666652
3 772.4170982819993 76.72623152726919 4.340277777781232e-05
5

KeyboardInterrupt: 

In [ ]:
# Plot
fig, axdict = logger.plot()
for key in axdict:
    if key not in ('xmean'):
        axdict[key].set_yscale('log')
plt.savefig(logger.prefix + '.pdf', tight_layout=True)

In [ ]:
# Obtained Design
x = ddcma.xmean
simulator = adaptf.simulator_list[adaptf.idx_current]
print("fidelity idx: {} (from 0 to {})".format(adaptf.idx_current, len(adaptf.simulator_list)-1))
design = simulator.ngnet(x)   
print(simulator.co(design)[:2])   # compliance, volume-fraction
simulator.co.plot(design, 'ngnet.pdf')

In [ ]:
!open ngnet.pdf